# fast postprocessing a list of trajectory folders
Tim Tyree<br>
9.30.2021

In [1]:
from lib.my_initialization import *
from lib import *
# from lib.measure.unwrap_and_smooth_cu import *
# from lib.routines.unwrap_and_smooth_trajectories_cu import *
from lib.rapids_func import *
import cupy as cp, cudf, numba.cuda as cuda
import rmm
# Switch to RMM allocator
cp.cuda.set_allocator(rmm.rmm_cupy_allocator)

In [2]:
# from lib.rapids_func.measure.annihilations_cu import routine_compute_radial_velocities_pbc_cu

In [3]:
# from lib.rapids_func.measure.annihilations_cu import *
# routine_compute_radial_velocities_pbc_cu

In [4]:
darkmode=False
if darkmode:
    # For darkmode plots
    from jupyterthemes import jtplot
    jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

# Postprocessing trajectories from the full models

In [5]:
input_fn_lst = [
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-fk-200x200/param_set_8_ds_5.0_tmax_10_diffCoef_0.0005/Log/ic200x200.0.2_traj_sr_400_mem_0.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.5/trajectories/ic002.11_traj_sr_600_mem_0.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-fk-200x200/param_set_8_ds_5.0_tmax_30_diffCoef_0.001_dt_0.025/trajectories/ic200x200.0.8_traj_sr_600_mem_0.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.025_Vthresh_-40/trajectories/ic002.22_traj_sr_600_mem_0.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.025/trajectories/ic001.33_traj_sr_600_mem_0.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/ds_5_param_qu_tmax_30_Ko_5.4_diffCoef_0.001/trajectories/ic001.22_traj_sr_400_mem_0.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ds_5_param_set_8_fastkernel_V_0.4/trajectories/ic_200x200.001.12_traj_sr_400_mem_2.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ds_5_param_set_8_fastkernel_V_0.5/trajectories/ic_200x200.001.21_traj_sr_400_mem_2.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ds_5_param_set_8_fastkernel_V_0.6/trajectories/ic_200x200.001.23_traj_sr_400_mem_2.csv'
]

In [6]:
# input_fn=search_for_file()
# input_fn_lst.append(input_fn)
# print(input_fn_lst)

In [7]:
for input_fn in input_fn_lst:
    df=pd.read_csv(input_fn)
    DT=np.around(get_DT(df),7)
    print(f"DT={DT} ms")
    retval=routine_postprocess_trajectory_folder(input_fn,DT,tavg1=4, tavg2=24,
                                            max_dtmax_thresh = 0,      #ms
                                            max_Rfinal_thresh = 0.2,   #cm
                                            min_duration_thresh = 40,  #ms
                                            npartitions=None,
                                            width=200,
                                            height=200,
                                            ds=5,
                                            use_drop_shorter_than=True,
                                            drop_shorter_than=40, #ms
                                            tmin=100., #ms
                                            pid_col='particle',
                                            t_col='t',
                                            printing=False)
    save_dir_lst,save_dir_lst2=retval

DT=0.4 ms
running return_moving_average_of_pbc_trajectories_and_save on 648 files...
the run time was 1.51 minutes.
the number of successfully smoothed trajectory files was 324
the run time was 0.89 minutes.
the number of trajectory files processed to annihilation files was 324
DT=0.5 ms
running return_moving_average_of_pbc_trajectories_and_save on 159 files...
the run time was 1.85 minutes.
the number of successfully smoothed trajectory files was 157
the run time was 0.69 minutes.
the number of trajectory files processed to annihilation files was 157
DT=0.025 ms
running return_moving_average_of_pbc_trajectories_and_save on 324 files...
the run time was 1.54 minutes.
the number of successfully smoothed trajectory files was 322
the run time was 1.20 minutes.
the number of trajectory files processed to annihilation files was 322
DT=0.025 ms
running return_moving_average_of_pbc_trajectories_and_save on 32 files...
the run time was 0.46 minutes.
the number of successfully smoothed trajecto

In [8]:
sus_lst=[1 for x in save_dir_lst if x is None]
sus_lst2=[1 for x in save_dir_lst2 if x is None]
print(sus_lst,sus_lst2)

[] []


# Postprocessing trajectories from the particle model

In [9]:
input_fn_lst = reversed([
    '/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_0_L_10_kappa_0_varkappa_0/Log/pbc_particle_log1_log.csv',
    '/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_0_L_10_kappa_1500_varkappa_5/Log/pbc_particle_log2_log.csv',
    '/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_2_L_10_kappa_0_varkappa_0/Log/pbc_particle_log5_log.csv',
    '/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_2_L_10_kappa_1500_varkappa_0/Log/pbc_particle_log5_log.csv',
    '/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_2_L_10_kappa_1500_varkappa_5/Log/pbc_particle_log5_log.csv'
])


# folder=f'/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results'
# input_fn=search_for_file(folder)
# input_fn_lst.append(input_fn)
# print(input_fn_lst)

In [10]:
pid_col='pid_explicit'
for input_fn in input_fn_lst:
    df=pd.read_csv(input_fn)
    DT=np.around(get_DT(df,pid_col=pid_col),7)
    print(f"DT={DT} ms")
    retval=routine_postprocess_trajectory_folder(input_fn,DT,tavg1=4, tavg2=14,
                                            max_dtmax_thresh = 0,      #ms
                                            max_Rfinal_thresh = 0.2,   #cm
                                            min_duration_thresh = 15,  #ms
                                            npartitions=None,
                                            width=10,
                                            height=10,
                                            ds=10,
                                            use_drop_shorter_than=False,
                                            drop_shorter_than=0, #ms
                                            tmin=0., #ms
                                            pid_col=pid_col,
                                            t_col='t',
                                            printing=False)
    save_dir_lst,save_dir_lst2=retval

DT=0.25 ms
running return_moving_average_of_pbc_trajectories_and_save on 200 files...
the run time was 0.20 minutes.
the number of successfully smoothed trajectory files was 200
the run time was 0.51 minutes.
the number of trajectory files processed to annihilation files was 200
DT=0.25 ms
running return_moving_average_of_pbc_trajectories_and_save on 200 files...
the run time was 0.28 minutes.
the number of successfully smoothed trajectory files was 200
the run time was 0.58 minutes.
the number of trajectory files processed to annihilation files was 200
DT=0.25 ms
running return_moving_average_of_pbc_trajectories_and_save on 200 files...
the run time was 0.30 minutes.
the number of successfully smoothed trajectory files was 200
the run time was 0.36 minutes.
the number of trajectory files processed to annihilation files was 200
DT=0.25 ms
running return_moving_average_of_pbc_trajectories_and_save on 200 files...
the run time was 0.21 minutes.
the number of successfully smoothed traject

In [11]:
sus_lst=[1 for x in save_dir_lst if x is None]
sus_lst2=[1 for x in save_dir_lst2 if x is None]
print(sus_lst,sus_lst2)

[] []


In [12]:
input_fn='/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_2_L_10_kappa_1500_varkappa_5/smoothed_trajectories_tavg_4/pbc_particle_log11_log_smoothed.csv'

In [13]:
routine_compute_radial_velocities_pbc_cu(input_fn,
                                     tavg2=14,
                                     width=10,  #for handling periodic boundary conditions (pbc)
                                     height=10, #for handling periodic boundary conditions (pbc)
                                     ds=10,     #domain size in cm.  set to width to have nulled effect
                                     max_dtmax_thresh = 0,      #ms
                                     max_Rfinal_thresh = 0.1,   #cm
                                     min_duration_thresh = 15,  #ms
                                     round_t_to_n_digits=7,
                                     trial_col='event_id_int',  #TODO: generalize kwarg handling of trial_col
                                     pid_col='pid_explicit',
                                     t_col='t',
                                     use_tavg2=True,
                                     save_df_pairs=True,
                                     printing=False,
                                     testing=True)

'/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_2_L_10_kappa_1500_varkappa_5/smoothed_annihilations_tavg1_4.0_tavg2_14.25_maxdtmax_0_maxRfin_0.1_mindur_15/pbc_particle_log11_log_smoothed_annihilations_denoised.csv'

In [14]:
pd.read_csv('/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_2_L_10_kappa_1500_varkappa_5/smoothed_annihilations_tavg1_4.0_tavg2_24.25_maxdtmax_0_maxRfin_0.1_mindur_25/pbc_particle_log11_log_smoothed_annihilations_denoised.csv')

,event_id_int,pid_self,pid_other,index,t,R,index_pairs,index_self,index_other,x_self,y_self,x_other,y_other,R_nosavgol,tdeath
0,111,3,2,350,0.00,4.124346,1,3,2,2.260345,3.690236,7.245564,2.464959,5.133587,1572.75
1,111,3,2,351,0.25,4.129769,1,19,18,2.255592,3.715496,7.246972,2.407531,5.159907,1572.50
2,111,3,2,352,0.50,4.135220,1,35,34,2.252711,3.728793,7.237700,2.419068,5.154172,1572.25
3,111,3,2,353,0.75,4.141164,1,51,50,2.284883,3.755425,7.246122,2.425278,5.136456,1572.00
4,111,3,2,354,1.00,4.148696,1,67,66,2.298194,3.766673,7.246516,2.454145,5.119435,1571.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6637,111,14,11,345,86.25,4.113905,0,3448,3447,9.499142,9.700434,9.393116,9.524863,0.205102,1.00
6638,111,14,11,346,86.50,4.114758,0,3454,3453,9.465365,9.660836,9.346534,9.521122,0.183413,0.75
6639,111,14,11,347,86.75,4.115191,0,3460,3459,9.421797,9.632707,9.396446,9.542200,0.093990,0.50
6640,111,14,11,348,87.00,4.117155,0,3466,3465,9.394736,9.559649,9.410317,9.576310,0.022811,0.25
